In [1]:
import os
os.environ['AOC_SESSION'] = '53616c7465645f5f37c74a4bd1a2394b4fa5dcbb449244bb183873f839c409c1c2b5d3191175e6de6040979aeb78f7894665a168392bc55c2d58d905f027b397'

In [2]:
import aocd
from aocd.models import Puzzle
day = 24
year = 2024
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [526]:
import operator
from collections import defaultdict
import random
from itertools import combinations
from statistics import median

In [5]:
test_data = """x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj"""
data_test = test_data.splitlines()

In [4]:
data = puzzle.input_data.splitlines()
len(data), data[:10]

(313,
 ['x00: 1',
  'x01: 0',
  'x02: 1',
  'x03: 1',
  'x04: 0',
  'x05: 0',
  'x06: 1',
  'x07: 1',
  'x08: 0',
  'x09: 1'])

In [19]:
operator.or_?

Signature: operator.or_(a, b, /)
Docstring: Same as a | b.
Type:      builtin_function_or_method


In [706]:
class Wire():
    def __init__(self, val=None):
        self.waiting = True
        self.val = None
        self.set(val)
    
    def set(self, val):
        if val is not None:
            self.val = val
            self.waiting = False
            
    def reset(self):
        self.val = None
        self.waiting = True
    
    def __repr__(self):
        return str(self.val)

class Gate():
    def __init__(self, in1, in2, out, op, index=-1):
        self.waiting = True
        self.wire_in1 = in1
        self.wire_in2 = in2
        self.wire_out = out
        self.wire_out_original = out
        self.operator_str = op
        self.index = index
        match op:
            case 'AND':
                self.operator = operator.and_
            case 'OR':
                self.operator = operator.or_
            case 'XOR':
                self.operator = operator.xor
        self.is_set1 = False
        self.is_set2 = False
        self.out = None
    
    def reset(self):
        self.is_set1 = False
        self.is_set2 = False
        self.out = None
        self.wire_out = self.wire_out_original
        self.waiting = True
        
    def set_in1(self, val):
        self.in1 = val
        self.is_set1 = True
        if self.is_set2:
            return self.compute()
    
    def set_in2(self, val):
        self.in2 = val
        self.is_set2 = True
        if self.is_set2:
            return self.compute()
    
    def compute(self):
        if self.is_set1 and self.is_set2:
            self.waiting = False
            self.out = self.operator(self.in1, self.in2)
            return self.out
        return None
            
    def __repr__(self):
        return f"{self.wire_in1} {self.operator_str} {self.wire_in2} --> {self.wire_out} : " + (str(self.out) if self.out is not None else "X")
        

In [212]:
def parse(wires, gates):
    wires = {wname: Wire(int(val)) for wname, val in map(lambda w: w.split(": "), wires.splitlines())}
    
    ans_gates = []
    for gate in gates.splitlines():
        # x00 AND y00 -> z00
        in1, op, in2, _ , out = gate.split(' ')
        ngate = Gate(in1, in2, out, op)
        if in1 in wires and not wires[in1].waiting:
            ngate.set_in1(wires[in1].val)
        if in2 in wires and not wires[in2].waiting:
            ngate.set_in2(wires[in2].val)

        ans_gates.append(ngate)
        if out not in wires:
            wires[out] = Wire()
        wires[out].set(ngate.compute())

    return wires, ans_gates

In [213]:
%%time
wires, gates = puzzle.input_data.split('\n\n')
# wires, gates = test_data.split('\n\n')
wires, gates = parse(wires, gates)

while True:
    todo = [gate for gate in gates if gate.waiting]
    # print(len(todo))
    if len(todo) == 0:
        break
    for gate in todo:
        # print(gate)
        if not wires[gate.wire_in1].waiting:
            gate.set_in1(wires[gate.wire_in1].val)
        if not wires[gate.wire_in2].waiting:
            gate.set_in2(wires[gate.wire_in2].val)
        res = gate.compute()
        if res is not None:
            # print(gate, res)
            wires[gate.wire_out].set(res)
        
    if all([wire.val is not None for wname, wire in wires.items() if wname.startswith('z')]):
        break
            
        

CPU times: user 9.57 ms, sys: 56 µs, total: 9.63 ms
Wall time: 7.86 ms


In [149]:
tpls = sorted([(wname, wire.val) for wname, wire in wires.items() if wname.startswith('z')])[::-1]
binary = "".join(str(val) for name, val in tpls)
res = int(binary, 2)
res

52728619468518

In [150]:
puzzle.answer_a = res

That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


In [151]:
# Part 2

In [407]:
test_data2 = """x00: 0
x01: 1
x02: 0
x03: 1
x04: 0
x05: 1
y00: 0
y01: 0
y02: 1
y03: 1
y04: 0
y05: 1

x00 AND y00 -> z05
x01 AND y01 -> z02
x02 AND y02 -> z01
x03 AND y03 -> z03
x04 AND y04 -> z04
x05 AND y05 -> z00"""

In [478]:
wires_raw, gates_raw = puzzle.input_data.split('\n\n')
# wires_raw, gates_raw = test_data2.split('\n\n')


In [728]:
def parse2(wires, gates):
    wires = {wname: Wire(int(val)) for wname, val in map(lambda w: w.split(": "), wires.splitlines())}
    
    ans_gates = []
    for i, gate in enumerate(gates.splitlines()):
        # x00 AND y00 -> z00
        in1, op, in2, _ , out = gate.split(' ')
        ngate = Gate(in1, in2, out, op, index=i)
        ans_gates.append(ngate)
    
        if out not in wires:
                wires[out] = Wire()

    return wires, ans_gates

def init_wires(a, b, gates, wires, Nbit=45):
    for name in wires:
        wires[name].reset()
        
    for i in range(Nbit):
        wires[f'x{i:02}'].set((a >> i) & 1)
        wires[f'y{i:02}'].set((b >> i) & 1)
        
    for gate in gates:
        gate.reset()
    return gates, wires

def compute(wires, gates, crosses=[], Nbit=45):
    steps = 0
    gates = gates[:]
    
    for cross in crosses:
        i, j = cross
        gates[i].wire_out, gates[j].wire_out = gates[j].wire_out, gates[i].wire_out
        # print(gates[i], gates[j])
    
    last_len_todo = 1e12
    while True:
        todo = [gate for gate in gates if gate.waiting]
        ntodo = len(todo)
        if not ntodo < last_len_todo:
            return
        last_len_todo = ntodo
        # print(len(todo))
        if ntodo == 0:
            break
        for gate in todo:
            # print(gate)
            if not wires[gate.wire_in1].waiting:
                gate.set_in1(wires[gate.wire_in1].val)
            if not wires[gate.wire_in2].waiting:
                gate.set_in2(wires[gate.wire_in2].val)
            res = gate.compute()
            if res is not None:
                # print(gate, res)
                wires[gate.wire_out].set(res)

        if all([wire.val is not None for wname, wire in wires.items() if wname.startswith('z')]):
            return [(wires[f'z{i:02}'].val) for i in range(Nbit, -1, -1)]
            
        if steps > 20:
            return None
        
def get_fault_bit(wires, gates, crosses=[], Nbit=45, nchecks=25, fault_bit=None):
    result = []
    for j in range(nchecks):
        a = random.randint(0, 1<<(Nbit-0))
        b = random.randint(0, 1<<(Nbit-0))
        c = a+b
        # wires, gates = parse2(wires_raw, gates_raw)
        gates, wires = init_wires(a, b, gates, wires)
        
        res = compute(wires, gates, crosses=crosses)
        if res is None:
            return 0
        add = int("".join(map(str, res)), 2)
        cmp = c ^ add
        i = 0
        while not cmp & 1:
            cmp >>= 1
            i += 1
            if i == Nbit:
                break
        if fault_bit is not None:
            if i <= fault_bit:
                return i
        result.append(i)  
    return min(result)

def get_back_connections(gates, Nbit=45):
    bcon = defaultdict(set)
    for i in range(Nbit):
        search = [f'z{i:02}']
        while len(search):
            wire_search = search.pop()
            nxt = [gate for gate in gates if gate.wire_out == wire_search]
            for g in nxt:
                bcon[i].add(g)
                search.append(g.wire_in1)
                search.append(g.wire_in2) 
    return bcon


def check_gate(igate, wires, gates, bcon, crosses=[], Nbit=45):
    fault_bit = get_fault_bit(wires, gates, crosses=crosses, Nbit=45, nchecks=5)
    # print(fault_bit)
    for i, gate in enumerate(gates):
        if i == igate:
            continue
        
        res = get_fault_bit(wires, gates, crosses=crosses+[(igate, i)], nchecks=3, fault_bit=fault_bit)
        if res > fault_bit:
            res = get_fault_bit(wires, gates, crosses=crosses+[(igate, i)], nchecks=25)
        if res > fault_bit:
            return res, i,
    return None, None

def find_crosses(wires, gates, bcon, Nbit=45):
    crosses = []
    fault_bit = get_fault_bit(wires, gates, Nbit=45, nchecks=15)
    while len(crosses) < 4:
        chk_gates = bcon[fault_bit]            
        for i in range(fault_bit):
            chk_gates -= bcon[i]
        print(f"checking: {chk_gates}")
        for gate in chk_gates:
            igate = gate.index
            new_fault_bit, jgate = check_gate(igate, wires, gates, bcon, crosses=crosses, Nbit=45)
            if new_fault_bit is not None and new_fault_bit > fault_bit:
                fault_bit = new_fault_bit
                crosses.append((igate, jgate))
                print(crosses, fault_bit)
                break
    return crosses

In [729]:
wires, gates = parse2(wires_raw, gates_raw)

In [730]:
%%time
bcons = get_back_connections(gates)
# check_gate(152, wires, gates, bcons, [])
crosses = find_crosses(wires, gates, bcons)
crosses

checking: {x12 AND y12 --> z12 : 0}
[(152, 197)] 19
checking: {mcc OR cgn --> jcg : 0, x19 XOR y19 --> wmb : 0, msb AND qkr --> mcc : 0, wmb AND jcg --> z19 : 0, y18 AND x18 --> cgn : 0}
[(152, 197), (145, 82)] 23
checking: {y23 AND x23 --> fgn : 0, fgn XOR shg --> z23 : 0, gtb AND hbj --> kjm : 0, ntn OR kjm --> shg : 0, y22 AND x22 --> ntn : 0}
[(152, 197), (145, 82), (5, 184)] 37
checking: {bfh OR hpn --> vhp : 0, gng OR qgb --> z37 : 1, vhp AND smb --> qgb : 0, x36 AND y36 --> bfh : 0, y37 XOR x37 --> smb : 0, x37 AND y37 --> gng : 1, rqh AND wvd --> hpn : 0}
[(152, 197), (145, 82), (5, 184), (120, 178)] 45
CPU times: user 15.2 s, sys: 0 ns, total: 15.2 s
Wall time: 15.2 s


[(152, 197), (145, 82), (5, 184), (120, 178)]

In [740]:
res = ",".join(sorted([gates[igate].wire_out for igate in sum(crosses, start=tuple())]))
res

'dck,fgn,nvh,qdg,vvf,z12,z19,z37'

In [660]:
puzzle.answer_b = res

That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 24! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


In [700]:
# analyse connections

con = {'x': defaultdict(set), 'y': defaultdict(set)}

for i in range(Nbit):
    for prefix in 'xy':
        search = [f'{prefix}{i:02}']
        while len(search):
            wire_search = search.pop()
            nxt = [gate for gate in gates if gate.wire_in1 == wire_search or gate.wire_in2 == wire_search]
            for g in nxt:
                con[prefix][i].add(g)
                search.append(g.wire_out)
# con
# all x and y bits are connected with all z bits of higher significance

In [3]:
# 4HbQ
from operator import xor as XOR, or_ as OR, and_ as AND

for l in open('data/input_24'):
    try:    a,x,b,_,c = l.split(); exec(f'{c}=lambda:{x}({a}(),{b}())')
    except: exec(l.replace(':', '=lambda:'))

print(sum(eval(f'z{i:02}()<<{i}') for i in range(46)))



lines = [l.split() for l in open('data/input_24') if '->' in l]

r = lambda c, y: any(y == x and c in (a, b) for a, x, b, _, _ in lines)

print(*sorted(c for a, x, b, _, c in lines if
    x == "XOR" and all(d[0] not in 'xyz' for d in (a, b, c)) or
    x == "AND" and not "x00" in (a, b) and r(c, 'XOR') or
    x == "XOR" and not "x00" in (a, b) and r(c, 'OR') or
    x != "XOR" and c[0] == 'z' and c != "z45"), sep=',')

52728619468518
dck,fgn,nvh,qdg,vvf,z12,z19,z37
